In [1]:
"""
自动下载 USGS 地震事件对应 IRIS 波形示例脚本
依赖: obspy
安装: pip install obspy requests
"""

import os
import requests
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

# 1️⃣ 配置
USGS_EVENTS_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"
OUTPUT_DIR = "waveforms"
STATIONS = ["IU", "CI", "ANMO"]  # 可指定台网
CHANNELS = "BH?"  # 广泛地震波道，如 BHZ, BHN, BHE
TIME_PADDING = 60  # 秒, 每个事件前后延长时间

# 创建目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 2️⃣ 获取 USGS 事件
print("Fetching USGS events...")
res = requests.get(USGS_EVENTS_URL)
res.raise_for_status()
data = res.json()
events = data.get("features", [])

print(f"Found {len(events)} events.")



Fetching USGS events...
Found 268 events.

Failed to download IU: Format "MSEED" is not supported. Supported types: 
Failed to download CI: Format "MSEED" is not supported. Supported types: 


KeyboardInterrupt: 

In [2]:
client = Client("IRIS")

for evt in events:
    props = evt["properties"]
    coords = evt["geometry"]["coordinates"]
    mag = props.get("mag")
    time = UTCDateTime(props["time"] / 1000)
    lat, lon, depth = coords[1], coords[0], coords[2]

    starttime = time - TIME_PADDING
    endtime = time + TIME_PADDING

    print(f"\nDownloading waveforms for event {props['title']}")

    for network in STATIONS:
        try:
            st = client.get_waveforms(
                network=network,
                station="*",
                location="*",
                channel=CHANNELS,
                starttime=starttime,
                endtime=endtime,
            )
            if st:
                # 按台站拆分保存
                for tr in st:
                    sta = tr.stats.station
                    net = tr.stats.network
                    fname = f"{props['id']}_{net}_{sta}_{tr.stats.starttime.date}.mseed"
                    path = os.path.join(OUTPUT_DIR, fname)
                    tr.write(path, format="MSEED")  # 单条 Trace 写入
                    print(f"Saved: {path}")
        except Exception as e:
            print(f"Failed to download {network}: {e}")

print("\nAll done!")


Failed to download IU: Format "MSEED" is not supported. Supported types: 
Failed to download CI: Format "MSEED" is not supported. Supported types: 


KeyboardInterrupt: 

In [6]:
# etas_example.py
import requests
import pandas as pd
import numpy as np
from mleetas import etas
import matplotlib.pyplot as plt

# -------------------------------
# 1️⃣ 下载 USGS 地震数据（近 30 天）
# -------------------------------
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "geojson",
    "starttime": "2025-10-24",  # 可改成最近30天
    "endtime": "2025-11-24",
    "minmagnitude": 2.5,
    "orderby": "time-asc",      # 时间升序
}
res = requests.get(url, params=params)
data = res.json()

# -------------------------------
# 2️⃣ 转成 DataFrame
# -------------------------------
events = []
for f in data['features']:
    props = f['properties']
    coords = f['geometry']['coordinates']
    events.append({
        "time": pd.to_datetime(props['time'], unit='ms'),
        "lat": coords[1],
        "lon": coords[0],
        "depth": coords[2],
        "mag": props['mag'],
    })

df = pd.DataFrame(events)
print(f"Loaded {len(df)} events")

# -------------------------------
# 3️⃣ ETAS 拟合
# -------------------------------
# 将时间转换为天（浮点数）
t0 = df['time'].min()
df['t'] = (df['time'] - t0).dt.total_seconds() / (24 * 3600)

# 使用 ETAS 包
# 默认参数：background_rate=0.01, K=0.5, alpha=1.5, c=0.01, p=1.1
model = etas.ETAS(t=df['t'].values, M=df['mag'].values)
model.fit()  # 拟合模型



# -------------------------------
# 4️⃣ 计算条件强度 λ(t)（每小时）
# -------------------------------
t_pred = np.linspace(df['t'].min(), df['t'].max() + 1, 200)  # 向前预测 1 天
lam = etas_model.conditional_intensity(t_pred)

# -------------------------------
# 5️⃣ 可视化
# -------------------------------
plt.figure(figsize=(12, 4))
plt.plot(t_pred, lam, label='ETAS λ(t)')
plt.scatter(df['t'], np.zeros_like(df['t']), color='red', marker='|', label='Events')
plt.xlabel('Time (days since start)')
plt.ylabel('Conditional intensity λ(t)')
plt.title('ETAS Model - Conditional Intensity')
plt.legend()
plt.show()

Loaded 1413 events


AttributeError: module 'mleetas.etas' has no attribute 'ETAS'